# Desarrollo de Servicios de Machine Learning con FastAPI

## 0. Preparación del entorno con conda


Antes de empezar necesitamos un **entorno aislado**: es un espacio lógico donde se guarda una instalación concreta de Python junto con todas las bibliotecas y versiones que requiere el proyecto. Mantener cada proyecto en su propio entorno evita conflictos entre dependencias (por ejemplo, dos proyectos que requieren versiones distintas de `scikit-learn`) y facilita reproducir resultados en otros equipos, ya que podemos describir exactamente qué paquetes están instalados.

**¿Conda o Anaconda?** `conda` es el gestor de paquetes y entornos multiplataforma; `Anaconda` es una distribución completa que ya incluye `conda`, Python y un conjunto grande de librerías científicas preinstaladas. Miniconda es la versión ligera: solo trae `conda` y Python, permitiéndote instalar únicamente lo que necesitas. Para este curso basta con Miniconda, porque reducimos descargas y tenemos control total sobre las dependencias.
1. **Descarga e instala Miniconda o Anaconda** siguiendo la [documentación oficial de conda](https://docs.conda.io/projects/miniconda/en/latest/). Elige el instalador que coincida con tu sistema operativo (Windows/macOS/Linux) y arquitectura (x86_64 o ARM). Durante la instalación marca la opción de añadir conda al `PATH` solo si sabes lo que implica; en Windows es preferible usar el Anaconda Prompt que se crea automáticamente.
2. **Verifica la instalación** abriendo una terminal nueva (PowerShell, CMD o Anaconda Prompt) y ejecutando `conda --version`. Si el comando no se reconoce, revisa las variables de entorno o vuelve a abrir la terminal para que se refresque el `PATH`.
3. **Actualiza conda** para asegurarte de tener los últimos parches de seguridad y compatibilidad:
```bash
conda update -n base -c defaults conda
```
4. **Crea un entorno limpio para este proyecto** con la versión de Python recomendada:
```bash
conda create -n ml-api python=3.11 -y
```
5. **Activa el entorno** y confirma que el prompt muestra `(ml-api)` al inicio:
```bash
conda activate ml-api
python --version
```
6. **Ubica la carpeta del proyecto** (donde está `train.py`). En Windows puedes usar `cd "ruta\al\proyecto"` desde la misma terminal activada. Este será el punto de partida para los pasos posteriores de entrenamiento y despliegue.

## 1. Introducción

En este notebook vamos a ver cómo desarrollar un **servicio de inferencia de Machine Learning** usando FastAPI.

Objetivos de la unidad:

- Entender cómo separar entrenamiento y servicio.
- Validar entradas de forma sencilla.
- Servir un modelo ML con FastAPI.
- Introducir conceptos de endpoints, batch inference y manejo de errores.


## 2. Entrenamiento del modelo



Primero entrenaremos un modelo de ML real, aunque muy simple, para poder usarlo en la API.

Nuestro caso de uso: **predicción de retraso de vuelos**.

Features:

- `distance`: distancia del vuelo en km.
- `bad_weather`: booleano indicando si hay mal tiempo.

Target:

- `delayed`: booleano, indica si el vuelo se retrasa.

Usaremos **Logistic Regression** con datos sintéticos.

Para ello vamos a ejecutar el fichero `train.py`


### Pasos para entrenar el modelo (entorno local)
1. **Activa el entorno `ml-api`** descrito en el paso 0 desde la terminal en la raíz del proyecto:
```bash
conda activate ml-api
```
2. **Instala las dependencias necesarias** usando el archivo [requirements.txt](requirements.txt) incluido en el repositorio:
```bash
pip install -r requirements.txt
```
   - Si añades nuevas librerías, actualiza el archivo con `pip freeze > requirements.txt` para mantener la lista sincronizada.
3. **Ejecuta el script de entrenamiento** y espera al mensaje de confirmación:
```bash
python train.py
```
4. **Comprueba que se generó el artefacto** `flight_delay_model.pkl` en la misma carpeta. Este es el archivo que luego cargará la API de FastAPI para servir predicciones.

**Notas importantes:**

- Estamos usando un modelo **real**, pero simple.
- Los datos son sintéticos, pero permiten probar la API.
- Guardamos el modelo con `joblib` para poder cargarlo luego en FastAPI.


## 3. Introducción a FastAPI

FastAPI es un framework ligero y moderno para crear APIs en Python.

Conceptos clave que veremos:

- `@app.post("/endpoint")` → crear un endpoint POST.
- `request.json()` → leer los datos enviados por el cliente.
- Validación manual de inputs (sin Pydantic en este ejemplo).
- Manejo de errores con `HTTPException`.
- Respuestas JSON.


## 4. Introducción a Pydantic



En entornos de producción, es crucial validar las entradas para:

- Evitar que el modelo falle.
- Asegurar consistencia de datos.
- Dar errores claros al cliente.

Pydantic es una librería esencial en el ecosistema de FastAPI que permite validar, parsear y serializar datos de manera automática y eficiente. Está basada en type hints de Python y ofrece una forma elegante de manejar datos entrantes y salientes en APIs.


#### ¿Qué es Pydantic?
- **Validación automática**: Convierte y valida datos JSON a tipos Python nativos.
- **Type hints**: Usa anotaciones de tipos para definir esquemas de datos.
- **Errores claros**: Proporciona mensajes de error detallados cuando los datos no cumplen las reglas.
- **Integración perfecta con FastAPI**: FastAPI usa Pydantic internamente para validar requests y responses.

#### Ventajas sobre validación manual
- **Menos código**: No necesitas escribir funciones de validación manual.
- **Documentación automática**: Genera schemas OpenAPI que aparecen en `/docs`.
- **Type safety**: Asegura que los datos tengan los tipos correctos.
- **Constraints**: Permite añadir restricciones como rangos, longitudes, etc.

#### Ejemplo básico de Pydantic
Veamos cómo se define un modelo Pydantic y cómo funciona:

In [6]:
from pydantic import BaseModel, Field

# Definir un modelo Pydantic
class FlightData(BaseModel):
    flight_id: str
    distance: int = Field(..., ge=100, le=5000, description="Distancia en km")
    bad_weather: bool

# Ejemplo de uso
data = {"flight_id": "ABC123", "distance": 1500, "bad_weather": False}
flight = FlightData(**data)
print(f"Vuelo: {flight.flight_id}, Distancia: {flight.distance} km, Mal tiempo: {flight.bad_weather}")

# Intentar con datos inválidos
try:
    invalid_data = {"flight_id": "XYZ", "distance": 6000, "bad_weather": "yes"}
    invalid_flight = FlightData(**invalid_data)
except Exception as e:
    print(f"Error de validación: {e}")

Vuelo: ABC123, Distancia: 1500 km, Mal tiempo: False
Error de validación: 1 validation error for FlightData
distance
  Input should be less than or equal to 5000 [type=less_than_equal, input_value=6000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


## 5. Endpoint predict

Este endpoint recibe los datos de un vuelo y devuelve:

- `flight_id`
- `delay_probability`
- `delayed` → True si la probabilidad > 0.5

Mostramos validación manual y manejo de errores.

**Qué enseñar:**

- Cómo recibir datos desde el cliente (`request.json()`).
- Validación manual sin Pydantic.
- Inferencia ML con datos validados.
- Manejo de errores HTTP.
- Respuesta JSON simple.


## 6. Endpoints didácticos adicionales


1. `/predict-batch` → múltiples predicciones
2. `/info` → información sobre la API
3. `/metrics` → contador de predicciones
4. `/simulate-error` → endpoint para mostrar manejo de errores

Estos endpoints son **útiles para ver cómo estructurar un servicio más completo**, aunque la lógica siga siendo simple.


## 7. Levantando la API y probando los endpoints


Ahora que tenemos el modelo entrenado y el código de la API comentado, vamos a ver cómo levantar el servicio y probar todos los endpoints.


### Requisitos previos:

- Modelo entrenado (`flight_delay_model.pkl` existe).
- Entorno `ml-api` activado.
- Dependencias instaladas: `pip install -r requirements.txt` (incluye FastAPI, Uvicorn y Requests).

### Levantar la API:

Ejecuta el siguiente comando en una terminal (desde la carpeta del proyecto, con entorno activado):

```bash
uvicorn app:app --reload --host 0.0.0.0 --port 8000
```

Esto iniciará el servidor en `http://localhost:8000`.

FastAPI genera automáticamente documentación interactiva en `http://localhost:8000/docs` (Swagger UI).

### ¿Qué es Uvicorn?


Uvicorn es un servidor web ASGI (Asynchronous Server Gateway Interface) ligero y rápido, diseñado específicamente para aplicaciones Python asíncronas como las creadas con FastAPI. ASGI es el estándar moderno para servidores web en Python, permitiendo manejar conexiones asíncronas de manera eficiente, lo que es ideal para APIs que necesitan alta concurrencia.

**¿Por qué Uvicorn en lugar de otros servidores?**
- Es rápido y eficiente para aplicaciones asíncronas.
- Soporta WebSockets, HTTP/2 y otras características modernas.
- Es el servidor recomendado por FastAPI.
- En desarrollo, permite recarga automática del código.

**Opciones comunes de Uvicorn:**
- `--reload`: Recarga automáticamente el servidor cuando cambias el código (útil en desarrollo).
- `--host`: Dirección IP donde escuchar (ej: `0.0.0.0` para todas las interfaces, `127.0.0.1` solo local).
- `--port`: Puerto donde escuchar (ej: `8000`).
- `--workers`: Número de procesos workers (para producción, aumenta concurrencia).
- `--log-level`: Nivel de logging (info, debug, warning, etc.).
- `--access-log`: Muestra logs de acceso HTTP.

Ejemplo avanzado para producción:
```bash
uvicorn app:app --host 0.0.0.0 --port 8000 --log-level info
```

### Uso Pydantic en FastAPI

En nuestro código `app.py`, usamos Pydantic para definir el modelo `FlightData`:

```python
class FlightData(BaseModel):
    flight_id: str
    distance: int = Field(..., ge=100, le=5000, description="Distancia del vuelo en km (100-5000)")
    bad_weather: bool
```

- **`BaseModel`**: Clase base de Pydantic.
- **`Field`**: Añade constraints y metadata (ge=100 significa >=100, le=5000 <=5000).
- **Type hints**: `str`, `int`, `bool` definen los tipos esperados.

En los endpoints:
```python
@app.post("/predict")
async def predict_delay(data: FlightData):
    # Pydantic ya validó 'data' automáticamente
    # Si los datos son inválidos, FastAPI retorna 422 Unprocessable Entity
```

#### Constraints comunes en Field


- `ge`, `le`: Greater/less than or equal (para números).
- `gt`, `lt`: Greater/less than.
- `min_length`, `max_length`: Para strings.
- `description`: Texto descriptivo que aparece en `/docs`.
- `default`: Valor por defecto.

#### Comparación: Manual vs Pydantic



**Validación manual (antes):**
- Código repetitivo.
- Errores personalizados con HTTPException.
- No genera docs automáticamente.

**Pydantic:**
- Declarativo y limpio.
- Validación automática + docs.
- Errores estandarizados.

Pydantic hace que las APIs sean más robustas y fáciles de mantener. ¡Es una de las razones por las que FastAPI es tan poderoso!

### Explorando la documentación automática en /docs


FastAPI genera automáticamente una interfaz interactiva de documentación usando Swagger UI, accesible en `http://localhost:8000/docs`.

**Qué puedes hacer en /docs:**
- **Ver todos los endpoints**: Lista completa de rutas, métodos HTTP y descripciones.
- **Probar endpoints**: Cada endpoint tiene un botón "Try it out" que abre un formulario para enviar requests directamente desde el navegador.
- **Ver esquemas**: Modelos de datos (request/response) en formato JSON Schema.
- **Autenticación**: Si la API tiene auth, puedes probarlo aquí.
- **Ejemplos**: FastAPI incluye ejemplos de requests basados en el código.

**Pasos para usar /docs:**
1. Levanta la API con Uvicorn.
2. Abre `http://localhost:8000/docs` en tu navegador.
3. Expande un endpoint (ej: POST /predict).
4. Haz clic en "Try it out".
5. Llena el JSON de ejemplo o modifica los parámetros.
6. Haz clic en "Execute" para enviar la request.
7. Verás la respuesta en "Server response".

También hay `/redoc` para una documentación alternativa más minimalista.

Esta documentación es generada automáticamente a partir del código FastAPI, por lo que siempre está actualizada con tus cambios.

### Probar /info


Este endpoint GET no requiere parámetros y devuelve información sobre la API.

Ejemplo con curl:
```bash
curl http://localhost:8000/info
```

Respuesta esperada:
```json
{
  "service": "Flight Delay ML API",
  "description": "API de ejemplo para enseñar FastAPI y ML",
  "version": "1.0",
  "features": ["predict", "predict-batch", "metrics", "simulate-error"]
}
```

In [1]:
import requests

# Probar el endpoint /info
response = requests.get("http://localhost:8000/info")
print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

Status Code: 200
Response JSON:
{'service': 'Flight Delay ML API', 'description': 'API de ejemplo para enseñar FastAPI y ML', 'version': '1.0', 'features': ['predict', 'predict-batch', 'metrics', 'simulate-error']}


### Probar /predict

Este endpoint POST recibe datos de un vuelo y devuelve la predicción.

Datos requeridos:
- `flight_id`: string (ej: "ABC123")
- `distance`: int entre 100 y 5000
- `bad_weather`: boolean

Ejemplo con curl:
```bash
curl -X POST "http://localhost:8000/predict" \
     -H "Content-Type: application/json" \
     -d '{"flight_id": "ABC123", "distance": 1500, "bad_weather": false}'
```

Respuesta esperada:
```json
{
  "flight_id": "ABC123",
  "delay_probability": 0.234,
  "delayed": false
}
```

Prueba con datos válidos e inválidos para ver validación.

In [2]:
# Probar el endpoint /predict con datos válidos
data = {
    "flight_id": "ABC123",
    "distance": 1500,
    "bad_weather": False
}
response = requests.post("http://localhost:8000/predict", json=data)
print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

# Probar con datos inválidos (distance fuera de rango)
data_invalid = {
    "flight_id": "XYZ789",
    "distance": 6000,  # Inválido: > 5000
    "bad_weather": True
}
response_invalid = requests.post("http://localhost:8000/predict", json=data_invalid)
print("\nStatus Code (inválido):", response_invalid.status_code)
print("Error Response:")
print(response_invalid.json())

Status Code: 200
Response JSON:
{'flight_id': 'ABC123', 'delay_probability': 0.2256581869248886, 'delayed': False}

Status Code (inválido): 400
Error Response:
{'detail': 'distance debe ser int entre 100 y 5000'}


### Probar /predict-batch


Este endpoint POST recibe una lista de vuelos y devuelve predicciones para todos.

Envía una lista de objetos con los mismos campos que /predict.

Ejemplo con curl:
```bash
curl -X POST "http://localhost:8000/predict-batch" \
     -H "Content-Type: application/json" \
     -d '[{"flight_id": "ABC123", "distance": 1500, "bad_weather": false}, {"flight_id": "DEF456", "distance": 3000, "bad_weather": true}]'
```

Respuesta esperada:
```json
{
  "predictions": [
    {"flight_id": "ABC123", "delay_probability": 0.234, "delayed": false},
    {"flight_id": "DEF456", "delay_probability": 0.678, "delayed": true}
  ],
  "count": 2
}
```

Los vuelos inválidos se ignoran silenciosamente.

In [3]:
# Probar el endpoint /predict-batch
batch_data = [
    {"flight_id": "ABC123", "distance": 1500, "bad_weather": False},
    {"flight_id": "DEF456", "distance": 3000, "bad_weather": True},
    {"flight_id": "GHI789", "distance": 500, "bad_weather": False}
]
response = requests.post("http://localhost:8000/predict-batch", json=batch_data)
print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

Status Code: 200
Response JSON:
{'predictions': [{'flight_id': 'ABC123', 'delay_probability': 0.2256581869248886, 'delayed': False}, {'flight_id': 'DEF456', 'delay_probability': 0.9746799425866687, 'delayed': True}, {'flight_id': 'GHI789', 'delay_probability': 0.09305330463338504, 'delayed': False}], 'count': 3}


### Probar /metrics


Este endpoint GET devuelve métricas de uso: total de predicciones y tiempo de actividad.

Ejemplo con curl:
```bash
curl http://localhost:8000/metrics
```

Respuesta esperada:
```json
{
  "total_predictions": 5,
  "uptime_seconds": 120
}
```

El contador aumenta con cada predicción exitosa.

In [4]:
# Probar el endpoint /metrics
response = requests.get("http://localhost:8000/metrics")
print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

Status Code: 200
Response JSON:
{'total_predictions': 4, 'uptime_seconds': 721}


### Probar /simulate-error


Este endpoint POST simula errores para testing. Envía `{"raise_error": true}` para provocar un error 418.

Ejemplo con curl (sin error):
```bash
curl -X POST "http://localhost:8000/simulate-error" \
     -H "Content-Type: application/json" \
     -d '{"raise_error": false}'
```

Respuesta:
```json
{"status": "ok", "message": "No se produjo error"}
```

Con error:
```bash
curl -X POST "http://localhost:8000/simulate-error" \
     -H "Content-Type: application/json" \
     -d '{"raise_error": true}'
```

Respuesta de error (418):
```json
{"detail": "Este es un error simulado para enseñar manejo"}
```

In [5]:
# Probar el endpoint /simulate-error sin error
data_ok = {"raise_error": False}
response_ok = requests.post("http://localhost:8000/simulate-error", json=data_ok)
print("Status Code (ok):", response_ok.status_code)
print("Response JSON:")
print(response_ok.json())

# Probar con error
data_error = {"raise_error": True}
response_error = requests.post("http://localhost:8000/simulate-error", json=data_error)
print("\nStatus Code (error):", response_error.status_code)
print("Error Response:")
print(response_error.json())

Status Code (ok): 200
Response JSON:
{'status': 'ok', 'message': 'No se produjo error'}

Status Code (error): 418
Error Response:
{'detail': 'Este es un error simulado para enseñar manejo'}


## Conclusiones



Has visto cómo:

1. **Levantar una API FastAPI** con un modelo ML cargado.
2. **Validar entradas** manualmente para asegurar integridad.
3. **Crear endpoints** para predicciones individuales y por lotes.
4. **Manejar errores** de forma apropiada.
5. **Probar la API** usando requests en Python o curl.

Recuerda:
- La documentación automática está en `http://localhost:8000/docs`.
- Para producción, considera usar Pydantic para validación

¡Experimenta modificando el código y probando diferentes escenarios!

## Ejercicios

### Ejercicio: Mini-API de clima de vuelo (facil)

**Objetivo:** practicar FastAPI, validacion con Pydantic y pruebas con `/docs` o `requests` sin complicar el modelo.

**Enunciado**

1. Crea un nuevo endpoint `POST /flight-status` que reciba un JSON con:
   - `flight_id` (string)
   - `distance` (int entre 100 y 5000)
   - `bad_weather` (bool)
2. Calcula un **indice de riesgo** simple con esta regla:

   $$
   \text{risk} = \frac{\text{distance}}{5000} + (1 \text{ si } \text{bad\_weather} \text{ es true, si no } 0)
   $$

3. Devuelve un JSON con:
   - `flight_id`
   - `risk_score` (numero entre 0 y 2, redondeado a 2 decimales)
   - `status` con estos valores:
     - `"low"` si `risk_score` < 0.8
     - `"medium"` si `risk_score` entre 0.8 y 1.2
     - `"high"` si `risk_score` > 1.2

4. Prueba el endpoint desde `/docs` o con `requests` usando 2 ejemplos:
   - uno con buen clima y distancia corta
   - otro con mal clima y distancia larga

**Opcional (si te sobra tiempo):**
- Agrega un endpoint `GET /health` que devuelva `{ "status": "ok" }`.
- Muestra un ejemplo de request y response en el propio docstring o en una celda markdown.